# PyTorch로 시작하는 딥러닝 입문
https://wikidocs.net/book/2788
## 01. CNN으로 MNIST 분류하기

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

/Users/limkahyun/opt/anaconda3/envs/sona/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 임의의 텐서 1x1x28x28 만들기
inputs = torch.Tensor(1, 1, 28, 28)
print(f'input shape : {inputs.shape}')

input shape : torch.Size([1, 1, 28, 28])


In [3]:
# 첫번재 합성곱층
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
# 두번째 합성곱층
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
# 맥스풀링
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [6]:
# 연결하기
out = conv1(inputs)
print('conv1:', out.shape)

out = pool(out)
print('maxpool1:', out.shape)

out = conv2(out)
print('conv2:', out.shape)

out = pool(out)
print('maxpool2:', out.shape)

conv1: torch.Size([1, 32, 28, 28])
maxpool1: torch.Size([1, 32, 14, 14])
conv2: torch.Size([1, 64, 14, 14])
maxpool2: torch.Size([1, 64, 7, 7])


In [7]:
# flatten
out = out.view(out.size(0), -1)
print('flatten:', out.shape)

flatten: torch.Size([1, 3136])


In [8]:
# 전결합층(Fully Connteced layer;fc) 통과
# 10은 분류 하고자 하는 갯수
fc = nn.Linear(3136, 10)
out = fc(out)
print('result:', out.shape)

result: torch.Size([1, 10])


In [9]:
# 파라미터 설정
torch.manual_seed(777)
lr = 0.001
n_epochs = 15
batch_size = 100

In [10]:
# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True,
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', 
                         train=False, 
                         transform=transforms.ToTensor(), 
                         download=True)

In [11]:
# 데이터 로더
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
# class로 모델 설계
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # input shape torch.Size([100, 1, 28, 28])
        # input shape (?, 28, 28, 1)
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                          # conv2d shape (?, 28, 28, 32)
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2)
                                          # pool shape (?, 14, 14, 32)
                                         )
        # inpust shape (?, 14, 14, 32)
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                          # conv2d shape (?, 14, 14, 64)
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2)
                                          # pool shape (?, 7, 7, 64)
                                         )
        # 전결합층 7x7x64
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
# 모델 정의
model = CNN()

# 비용함수, 옵티마이저 정의
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# 학습
for epoch in range(n_epochs):
    avg_cost = 0
    
    for X, Y in data_loader: # X는 미니배치, Y는 레이블
        
        # 가설
        hypothesis = model(X)
        
        # loss
        loss = cost(hypothesis, Y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # avg_cost
        avg_cost += loss / len(data_loader)
        
    print(f'Epoch {epoch}/{n_epochs}\tCost {avg_cost:.4f}')

In [15]:
# 테스트
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float()
    y_test = mnist_test.test_labels
    
    pred = model(X_test)
    correct = torch.argmax(pred, 1) == y_test
    accuracy = correct.float().mean()
    print('Accuracy:', accuracy.item())

torch.Size([10000, 1, 28, 28])


/Users/limkahyun/opt/anaconda3/envs/sona/lib/python3.8/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/limkahyun/opt/anaconda3/envs/sona/lib/python3.8/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.986299991607666


## 2. 깊은 CNN으로 MNIST 분류하기

In [16]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [17]:
# 파라미터 설정
lr = 0.001
n_epochs = 15
batch_size = 100

In [18]:
# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True
                         )
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True
                        )

In [19]:
# 데이터로더 정의
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True
                                         )

In [20]:
# 모델 설계
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5  # keep_prob 일부만 학습시킴 (keep_prob=1 전체 학습)
        
        # input shape (?, 28, 28, 1)
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                          # shape (?, 28, 28, 32)
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2)
                                          # shape (?, 14, 14, 32)
                                         )
        # input shape (?, 14, 14, 32)
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                          # shape (?, 14, 14, 64)
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2)
                                          # shape (?, 7, 7, 64)
                                         )
        # input shape (?, 7, 7, 64)
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                          # shape (?, 7, 7, 128)
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2)
                                          # shape (?, 3, 3, 128)
                                         )
        # flatten (size(0), 3*3*128), output 625
        self.fc1 = torch.nn.Linear(3*3*128, 625, bias=True)
        
        # 초기화
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        # input (?, 3, 3, 128)
        self.layer4 = torch.nn.Sequential(self.fc1,
                                          # flatten shape (?, 625)
                                          torch.nn.ReLU(),
                                          torch.nn.Dropout(p=1-self.keep_prob)
                                         )
        # input (?, 625) outout (?, 10(mnist분류갯수))
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [21]:
# 모델 정의
model = CNN()

# 비용함수, 옵티마이저
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [22]:
# 총배치 및 총 훈련데이터
total_batch = len(data_loader)
print('total batch:', total_batch)
print('total train data:', total_batch * batch_size)

total batch: 600
total train data: 60000


In [23]:
# 모델 학습
for epoch in range(n_epochs):
    avg_cost = 0
    
    for X, y in data_loader: # X 미니배치, y 레이블
        
        # model
        h = model(X)
        
        # loss
        loss = cost(h, y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_cost += loss/total_batch
        
    print(f'Epoch {epoch}/{n_epochs}\tCost {avg_cost:.6f}')

KeyboardInterrupt: 

In [ ]:
# 테스트
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float()
    y_test = mnist_test.test_labels
    
    pred = model(X_test)
    correct = torch.argmax(pred, 1) == y_test
    accuracy = correct.float().mean()
    print('Accuracy:', accuracy.item())